In [1]:
import hsbalance as hs

Using the Community license in this session. If you have a full Xpress license, first set the XPAUTH_PATH environment variable to the full path to your license file, xpauth.xpr, and then restart Python. If you want to use the FICO Community license and no longer want to see this message, set the XPAUTH_PATH environment variable to: /home/maged/anaconda3/envs/balance/lib/python3.8/site-packages/xpress/license/community-xpauth.xpr
NB: setting XPAUTH_PATH will also affect any other Xpress products installed on your system.


# Introduction

This notebook discuss how the `hsbalance` package handles ill-conditioned planes problem

Darlow discussed in his paper `Darlow, M. S. "The identification and elimination of non-independent balance planes in influence coefficient balancing." Turbo Expo: Power for Land, Sea, and Air. Vol. 79603. American Society of Mechanical Engineers, 1982.` three test cases can be found in test cases in `/test/`

## Parameters

In [2]:
ALPHA_math=[
                            ['1.41@45', '3.61@34', '3.61@34'],
                            ['3.16@72', '2.24@27', '2.24@27'],
                            ['2.83@45', '5@37', '5@37'],
                            ['3.16@18', '3.61@34', '4.47@27']]
A_math=[
                            ['3.16@72'], 
                            ['3.16@18'],
                            ['4.12@14'],
                            ['5.39@68']]

Convert to complex numbers (cartesian) form

In [3]:
A = hs.convert_matrix_to_cart(A_math)
ALPHA = hs.convert_matrix_to_cart(ALPHA_math)
# A, ALPHA

Adding ALPHA

In [4]:
alpha = hs.Alpha()
alpha.add(direct_matrix=ALPHA)

In [5]:
alpha.check()

Not a square matrix --> no exact solution
Ill-conditioned found in plane # [1]


A warning is raised that plane[1] (the second plane, remember python starts counting from 0) is ill-conditioned.  
Ill-conditioned planes mean that they are algebraically depending on other planes. It means the ill-conditioned plane carries not new information to the system. When solving(especially with least squares method) the system tend to infiltrate (adding excessive masses at these planes)

## Solving with Least squares:

In [6]:
model_LeastSquares = hs.LeastSquares(A, alpha, name='Least_squares') # Instantiate least square model

In [7]:
W_LeastSquares = model_LeastSquares.solve() #solve

In [8]:
hs.convert_matrix_to_math(W_LeastSquares)

array([['0.875 @ 99.4'],
       ['4.777 @ 98.0'],
       ['5.137 @ 271.1']], dtype='<U13')

In [9]:
residuals_LeastSquares = model_LeastSquares.expected_residual_vibration()
hs.convert_matrix_to_math(residuals_LeastSquares) # Expected residule vibrations

array([['1.638 @ 124.2'],
       ['0.46 @ 180.4'],
       ['1.288 @ 315.4'],
       ['0.0 @ 238.0']], dtype='<U13')

Root mean square error:

In [10]:
rmse_LeastSquares = model_LeastSquares.rmse()
rmse_LeastSquares

0.8464

### Discussion
The second plane is ill-conditioned as seen from the check, caused the second and third planes to increase the masses excessively.

In [11]:
alpha.check(ill_condition_remove=True)

Not a square matrix --> no exact solution
Ill-conditioned found in plane # [1]
Removing Ill-conditioned plane # [1]
IC matrix before removing[['1.41 @ 45.0' '3.61 @ 34.0' '3.61 @ 34.0']
 ['3.16 @ 72.0' '2.24 @ 27.0' '2.24 @ 27.0']
 ['2.83 @ 45.0' '5.0 @ 37.0' '5.0 @ 37.0']
 ['3.16 @ 18.0' '3.61 @ 34.0' '4.47 @ 27.0']]
IC matrix after removing[['1.41 @ 45.0' '3.61 @ 34.0']
 ['3.16 @ 72.0' '2.24 @ 27.0']
 ['2.83 @ 45.0' '5.0 @ 37.0']
 ['3.16 @ 18.0' '4.47 @ 27.0']]


by turning ill_condition_remove boolean to True we remove the second plane (ill-conditioned)

In [12]:
alpha.value

array([[0.99702056+0.99702056j, 2.99282564+2.01868638j],
       [0.9764937 +3.00533859j, 1.99585461+1.01693872j],
       [2.00111219+2.00111219j, 3.99317755+3.00907512j],
       [3.00533859+0.9764937j , 3.98279916+2.02933753j]])

## Solving with Least squares:

In [13]:
model_LeastSquares = hs.LeastSquares(A, alpha, name='Least_squares') # Instantiate least square model

In [14]:
W_LeastSquares = model_LeastSquares.solve() #solve

In [15]:
hs.convert_matrix_to_math(W_LeastSquares)

array([['0.524 @ 44.4'],
       ['1.137 @ 204.5']], dtype='<U13')

Remember those are weights for first and third planes as plane 2 has been removed by check.

In [16]:
residuals_LeastSquares = model_LeastSquares.expected_residual_vibration()
hs.convert_matrix_to_math(residuals_LeastSquares) # Expected residule vibrations

array([['1.186 @ 168.2'],
       ['0.826 @ 34.3'],
       ['2.835 @ 297.3'],
       ['2.514 @ 98.7']], dtype='<U13')

Root mean square error:

In [17]:
rmse_LeastSquares = model_LeastSquares.rmse()
rmse_LeastSquares

1.8401

### Discussion
by removing the second the correction weights at plane 3 has reduced to less than 4 times (from 5.14 kg to 1.14 kg). This causes a rise of RMSE and residual vibration on the other hand.   
It is left to the engineer judgment weather to remove (or constrain) the ill-conditioned planes or leave it to get the best RMSE possible.  
The target of this package is to give the engineer the best possible consulting service.